In [ ]:
# %load_ext autoreload
# %autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['figure.titlesize'] = 30
mpl.rcParams.update({"axes.grid" : True})

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

In [ ]:
tfList=[10,12,14]

basepath='/p/project/pines/li47/code/projectData/discNJN/'
ensembles=['cA2.09.48']
# ensembles=['cB211.072.64']
# ensembles=['cC211.06.80']
# ensembles=['cD211.054.96']


for ens in ensembles:
    with h5py.File(basepath+ens+'/data_merge/N.h5') as fN, h5py.File(basepath+ens+'/data_merge/N-j.h5') as fNj:
        
        moms=fN['mvec']
        momDicN={}
        for i,mom in enumerate(moms):
            momDicN[tuple(mom)]=i

        moms=fNj['mvec']
        momDicNj={}
        for i,mom in enumerate(moms):
            momDicNj[tuple(mom)]=i

        cfgs=list(fN['data'].keys())


        def getdata():
            p0=[0,0,0]
            pf1=[0,0,0]
            t_2pt=[fN['data'][cfg]['p,p'][:,momDicN[tuple(pf1 + p0 + p0 + pf1 + p0)],0] for cfg in cfgs]

            return t_2pt

        dat=getdata()

        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            t=np.log(t/np.roll(t,-1,axis=0))
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)

        fig, axs = yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)

        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        irow=0; icol=0
        tmin=1; tmax=20
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color='r',fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,2))

In [ ]:
rela1={}

tfList=[10,12,14]

(Gc,iso,preFactor)=(5,'j-',1)
# (Gc,iso,preFactor)=(0,'j+',1)

for ens in ensembles:
    with h5py.File(basepath+ens+'/data_merge/N.h5') as fN, \
        h5py.File(basepath+ens+'/data_merge/N-j.h5') as fNj, h5py.File(basepath+ens+'/data_merge/N-jbw.h5') as fNjb:
        
        
        moms=fN['mvec']
        momDicN={}
        for i,mom in enumerate(moms):
            momDicN[tuple(mom)]=i

        moms=fNj['mvec']
        momDicNj={}
        for i,mom in enumerate(moms):
            momDicNj[tuple(mom)]=i

        cfgs=list(fN['data'].keys())

        def getdata():
            z3=[0,0,0]
            pc=[0,0,1]; pcN=list(-np.array(pc))
            t_3pt=np.array([np.array([
                (
                    (
                        +(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,0]
                        -fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,5])

                        # -np.conj(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,0]
                        # -fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,5])

                        # -(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,0]
                        # -fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,5])

                        # +np.conj(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,0]
                        # -fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,5])


                        # +(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,10]
                        # -fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,15])

                        # -np.conj(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,10]
                        # -fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,15])

                        # -(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,10]
                        # -fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,15])

                        # +np.conj(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,10]
                        # -fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,15])
                    )
                    -
                    (0
                        # +(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,0]
                        # -fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,5])

                        # -np.conj(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,0]
                        # -fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,5])

                    #     -(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,0]
                    #     -fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,5])

                        # +np.conj(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,0]
                        # -fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,5])


                    #     +(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,10]
                    #     -fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,15])

                    #     -np.conj(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,10]
                    #     -fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,15])

                        # -(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,10]
                        # -fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,15])

                        # +np.conj(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,10]
                        # -fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,15])
                    )
        
                )/(2*1)
                 for tf in tfList],dtype=object)
             for cfg in cfgs])
            return t_3pt
 
        dat=getdata()

        def tFunc(dat):
            t=np.mean(dat,axis=0)
            def getRes(i_tf):
                tf=tfList[i_tf]
                return np.real(preFactor*t[i_tf][1:tf])
            t=np.array([getRes(i_tf) for i_tf in range(len(tfList))],dtype=object)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)

        fig, axs = yu.getFigAxs(1,1,Lrow=4*1.5,Lcol=5*1.5)

        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        irow=0; icol=0
        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print(np.abs(tErr/tMean))
            rela1[tf]=np.abs(tErr/tMean)
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],mfc='white')


In [ ]:
rela2={}

tfList=[10,12,14]

(Gc,iso,preFactor)=(5,'j-',1)
# (Gc,iso,preFactor)=(0,'j+',1)

for ens in ensembles:
    with h5py.File(basepath+ens+'/data_merge/N.h5') as fN, \
        h5py.File(basepath+ens+'/data_merge/N-j.h5') as fNj, h5py.File(basepath+ens+'/data_merge/N-jbw.h5') as fNjb:
        
        
        moms=fN['mvec']
        momDicN={}
        for i,mom in enumerate(moms):
            momDicN[tuple(mom)]=i

        moms=fNj['mvec']
        momDicNj={}
        for i,mom in enumerate(moms):
            momDicNj[tuple(mom)]=i

        cfgs=list(fN['data'].keys())

        def getdata():
            z3=[0,0,0]
            pc=[0,0,1]; pcN=list(-np.array(pc))
            t_3pt=np.array([np.array([
                (
                    (
                        +(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,0]
                        -fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,5])

                        # -np.conj(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,0]
                        # -fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,5])

                        # -(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,0]
                        # -fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,5])

                        # +np.conj(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,0]
                        # -fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,5])


                        # +(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,10]
                        # -fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,15])

                        # -np.conj(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,10]
                        # -fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,15])

                        # -(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,10]
                        # -fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,15])

                        # +np.conj(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,10]
                        # -fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,15])
                    )
                    -
                    (
                        # +(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,0]
                        # -fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,5])

                        -np.conj(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,0]
                        -fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,5])

                    #     -(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,0]
                    #     -fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,5])

                        # +np.conj(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,0]
                        # -fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,5])


                    #     +(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,10]
                    #     -fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,15])

                    #     -np.conj(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,10]
                    #     -fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,15])

                        # -(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,10]
                        # -fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,15])

                        # +np.conj(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,10]
                        # -fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,15])
                    )
        
                )/(2*1)
                 for tf in tfList],dtype=object)
             for cfg in cfgs])
            return t_3pt
 
        dat=getdata()

        def tFunc(dat):
            t=np.mean(dat,axis=0)
            def getRes(i_tf):
                tf=tfList[i_tf]
                return np.real(preFactor*t[i_tf][1:tf])
            t=np.array([getRes(i_tf) for i_tf in range(len(tfList))],dtype=object)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)

        fig, axs = yu.getFigAxs(1,1,Lrow=4*1.5,Lcol=5*1.5)

        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        irow=0; icol=0
        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print(np.abs(tErr/tMean))
            rela2[tf]=np.abs(tErr/tMean)
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],mfc='white')


In [ ]:
rela8={}

tfList=[10,12,14]

(Gc,iso,preFactor)=(5,'j-',1)
# (Gc,iso,preFactor)=(0,'j+',1)

pol=-1

for ens in ensembles:
    with h5py.File(basepath+ens+'/data_merge/N.h5') as fN, \
        h5py.File(basepath+ens+'/data_merge/N-j.h5') as fNj, h5py.File(basepath+ens+'/data_merge/N-jbw.h5') as fNjb:
        
        
        moms=fN['mvec']
        momDicN={}
        for i,mom in enumerate(moms):
            momDicN[tuple(mom)]=i

        moms=fNj['mvec']
        momDicNj={}
        for i,mom in enumerate(moms):
            momDicNj[tuple(mom)]=i

        cfgs=list(fN['data'].keys())

        def getdata():
            z3=[0,0,0]
            pc=[0,0,1]; pcN=list(-np.array(pc))
            t_3pt=np.array([np.array([
                (
                    (
                        +(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,0]
                        +pol*fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,5])

                        # -np.conj(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,0]
                        # +pol*fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,5])

                        -(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,0]
                        +pol*fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,5])

                        # +np.conj(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,0]
                        # +pol*fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,5])


                        +(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,15])

                        # -np.conj(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,10]
                        # +pol*fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,15])

                        -(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,15])

                        # +np.conj(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,10]
                        # +pol*fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,15])
                    )
                    -
                    (
                        +(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,0]
                        +pol*fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,5])

                        # -np.conj(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,0]
                        # +pol*fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,5])

                        -(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,0]
                        +pol*fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,5])

                        # +np.conj(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,0]
                        # +pol*fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,5])


                        +(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,15])

                        # -np.conj(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,10]
                        # +pol*fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,15])

                        -(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,15])

                        # +np.conj(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,10]
                        # +pol*fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,15])
                    )
        
                )/(2*8)
                 for tf in tfList],dtype=object)
             for cfg in cfgs])
            return t_3pt
 
        dat=getdata()

        def tFunc(dat):
            t=np.mean(dat,axis=0)
            def getRes(i_tf):
                tf=tfList[i_tf]
                return np.real(preFactor*t[i_tf][1:tf])
            t=np.array([getRes(i_tf) for i_tf in range(len(tfList))],dtype=object)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)

        fig, axs = yu.getFigAxs(1,1,Lrow=4*1.5,Lcol=5*1.5)

        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        irow=0; icol=0
        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print(np.abs(tErr/tMean))
            rela8[tf]=np.abs(tErr/tMean)
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],mfc='white')


In [ ]:
rela16={}

tfList=[10,12,14]

(Gc,iso,preFactor)=(5,'j-',1)
# (Gc,iso,preFactor)=(0,'j+',1)

pol=-1

for ens in ensembles:
    with h5py.File(basepath+ens+'/data_merge/N.h5') as fN, \
        h5py.File(basepath+ens+'/data_merge/N-j.h5') as fNj, h5py.File(basepath+ens+'/data_merge/N-jbw.h5') as fNjb:
        
        
        moms=fN['mvec']
        momDicN={}
        for i,mom in enumerate(moms):
            momDicN[tuple(mom)]=i

        moms=fNj['mvec']
        momDicNj={}
        for i,mom in enumerate(moms):
            momDicNj[tuple(mom)]=i

        cfgs=list(fN['data'].keys())

        def getdata():
            z3=[0,0,0]
            pc=[0,0,1]; pcN=list(-np.array(pc))
            t_3pt=np.array([np.array([
                (
                    (
                        # +(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,0]
                        # +pol*fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,5])

                        # -np.conj(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,0]
                        # +pol*fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,5])

                        # -(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,0]
                        # +pol*fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,5])

                        # +np.conj(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,0]
                        # +pol*fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,5])


                        +(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,15])

                        -np.conj(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,15])

                        -(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pc+pc+z3)],Gc,15])

                        +np.conj(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pc+z3+pcN+z3+z3)],Gc,15])
                    )
                    -
                    (0
                        +(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,0]
                        +pol*fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,5])

                        -np.conj(fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,0]
                        +pol*fNj['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,5])

                        -(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,0]
                        +pol*fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,5])

                        +np.conj(fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,0]
                        +pol*fNj['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,5])


                        +(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,15])

                        -np.conj(fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['p,'+iso+',p_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,15])

                        -(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(z3+z3+pcN+pcN+z3)],Gc,15])

                        +np.conj(fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,10]
                        +pol*fNjb['data'][cfg]['n,'+iso+',n_deltat_'+str(tf)][:,momDicNj[tuple(pcN+z3+pc+z3+z3)],Gc,15])
                    )
        
                )/(2*16)
                 for tf in tfList],dtype=object)
             for cfg in cfgs])
            return t_3pt
 
        dat=getdata()

        def tFunc(dat):
            t=np.mean(dat,axis=0)
            def getRes(i_tf):
                tf=tfList[i_tf]
                return np.real(preFactor*t[i_tf][1:tf])
            t=np.array([getRes(i_tf) for i_tf in range(len(tfList))],dtype=object)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)

        fig, axs = yu.getFigAxs(1,1,Lrow=4*1.5,Lcol=5*1.5)

        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        irow=0; icol=0
        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print(np.abs(tErr/tMean))
            rela16[tf]=np.abs(tErr/tMean)
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],mfc='white')


Some rotation (exact symm.) has the same effect with parity (broken symm.) in this case \
Parity is exact in this case \
Flavor swtiching (=the rotation*(parity*flavor)) as well

2 for flavors: p and n \
2 for parity: NJ(q)N(q) and NJ(-q)N(-q) \
2 for time reversal: forward and backward \
2 for complex conjugation: NJ(q)N(q) and N(q)J(-q)N

No symmetry breaking effect in this case?

In [ ]:
def run(relaA,relaB):
    for tf in tfList:
        print((relaA[tf]/relaB[tf])**2)
        
print('1->16')
run(rela1,rela16)

print('1->8: w.o. complex')
run(rela1,rela8)

print('8->16')
run(rela8,rela16)

# print('1->2:')
# run(rela1,rela2)